In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scqubits as scq
import pandas as pd

In [ ]:
qub_name = "S001"

loadpath = f"../result/{qub_name}/params.csv"
params_df = pd.read_csv(loadpath)

cflx, period = params_df.loc[0, ["Half flux", "Period"]]
params = params_df.loc[0, ["EJ", "EC", "EL"]].values


def phi2mA(ratio):
    return (period * (ratio - 0.5) + cflx) * 1e3


def mA2phi(current):
    return (current / 1e3 - cflx) / period + 0.5


params_df.head()

In [ ]:
freqs_path = f"../result/{qub_name}/freqs.csv"
freqs_df = pd.read_csv(freqs_path)

s_mAs = freqs_df["Current (mA)"]
s_fpts = freqs_df["Freq (MHz)"]
s_T1s = freqs_df["T1 (μs)"]
s_T1errs = freqs_df["T1err (μs)"]

freqs_df.head()

In [ ]:
# sort by current
s_mAs, s_fpts, s_T1s, s_T1errs = zip(
    *sorted(zip(s_mAs, s_fpts, s_T1s, s_T1errs), reverse=True)
)
s_mAs, s_fpts, s_T1s, s_T1errs = (
    np.array(s_mAs),
    np.array(s_fpts),
    np.array(s_T1s),
    np.array(s_T1errs),
)
s_flxs = mA2phi(s_mAs)

In [ ]:
fig, ax = plt.subplots(2, 1, constrained_layout=True)
ax[0].set_title(r"$T_1$ as function of current")
ax[0].set_xlabel(r"Current (mA)")
ax[0].set_ylabel(r"$T_1$ ($\mu s$)")
ax[0].errorbar(s_mAs, s_T1s, yerr=s_T1errs, fmt=".-")
ax[0].grid()

ax[1].set_title(r"$T_1$ as function of flux")
ax[1].set_xlabel(r"flux quanta $\phi_{ext}/\phi_0$")
ax[1].set_ylabel(r"$T_1$ ($\mu s$)")
ax[1].errorbar(s_flxs, s_T1s, yerr=s_T1errs, fmt=".-")
ax[1].grid()
fig.savefig(f"../result/{qub_name}/T1s.png")
plt.show()

In [ ]:
scq.settings.T1_DEFAULT_WARNING = False
fluxonium = scq.Fluxonium(*params, flux=0.5, cutoff=30, truncated_dim=5)
hilbertspace = scq.HilbertSpace([fluxonium])

In [ ]:
fig, ax = fluxonium.plot_coherence_vs_paramvals(
    param_name="flux",
    param_vals=np.linspace(0.0, 0.5, 501),
    common_noise_options=dict(i=1, j=0, T=0.15),
    noise_channels=[("t1_capacitive", dict(Q_cap=4e6))],
)

In [ ]:
Temp = 0.25
Q_ind = 2.5e6
Q_cap = 1.5e5
x_qp = 8e-6

flxs = np.linspace(0.0, 0.5, 501)
T1_eff_fig, T1_eff_ax = fluxonium.plot_t1_effective_vs_paramvals(
    param_name="flux",
    param_vals=flxs,
    xlim=([flxs.min(), flxs.max()]),
    common_noise_options=dict(i=1, j=0, T=Temp),
    noise_channels=[
        ("t1_inductive", dict(Q_ind=Q_ind)),
        ("t1_quasiparticle_tunneling", dict(x_qp=x_qp)),
        ("t1_capacitive", dict(Q_cap=Q_cap)),
    ],
)
T1_eff_ax.errorbar(s_flxs, s_T1s * 1e3, yerr=s_T1errs * 1e3, fmt=".-")
T1_eff_ax.set_ylim([s_T1s.min() * 5e2, s_T1s.max() * 2e3])
_, T1_eff = T1_eff_ax.lines[0].get_data()
plt.show()

In [ ]:
T1_cap = []
T1_ind = []
T1_qp = []
for flux in flxs:
    fluxonium.flux = flux
    T1_cap.append(fluxonium.t1_capacitive(T=Temp, Q_cap=Q_cap) * 1e-9)
    T1_ind.append(fluxonium.t1_inductive(T=Temp, Q_ind=Q_ind) * 1e-9)
    T1_qp.append(fluxonium.t1_quasiparticle_tunneling(T=Temp, x_qp=x_qp) * 1e-9)

In [ ]:
fig, ax = plt.subplots()
ax.set_yscale("log")
ax.plot(flxs, T1_cap, label="Capacitive")
ax.plot(flxs, T1_ind, label="Inductive")
ax.plot(flxs, T1_qp, label="Quasi particle")
ax.plot(flxs, T1_eff * 1e-9, label="Effective")
ax.errorbar(
    s_flxs, s_T1s * 1e-6, yerr=s_T1errs * 1e-6, fmt=".-", label="Data", color="purple"
)
ax.set_ylim([s_T1s.min() * 5e-7, s_T1s.max() * 2e-5])
ax.set_title(f"Q_cap = {Q_cap:.1e}, Q_ind = {Q_ind:.1e}, x_qp = {x_qp:.1e}")
ax.grid()
ax.legend()
fig.savefig(f"../result/{qub_name}/T1_fit.png")
plt.show()